In [ ]:
import SimpleITK as sitk

mandible = sitk.ReadImage(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_totalseg.nii.gz")

mandible_bin = sitk.Cast(mandible > 0, sitk.sitkUInt8)

mandible_filled = sitk.BinaryFillhole(mandible_bin, fullyConnected=True)

mandible_smoothed = sitk.BinaryMorphologicalClosing(mandible_filled, [2,2,2])

sitk.WriteImage(mandible_smoothed, r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_totalseg_filled.nii.gz")

print("Done")